In [4]:
from ultralytics import YOLO
import cv2
import numpy as np

def detect_fruits(img_path, model, view):
    """Detect fruits and return their centers and dominant color along with view."""
    results = model.predict(img_path)[0]
    fruits = []
    img = cv2.imread(img_path)
    
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        center = ((x1 + x2) // 2, (y1 + y2) // 2)
        fruit_roi = img[y1:y2, x1:x2]
        color = detect_color(fruit_roi)
        
        fruits.append({
            "view": view,
            "bbox": (x1, y1, x2, y2),
            "color": color
        })
    return fruits

def detect_color(roi):
    """Detect the dominant color of the fruit using refined HSV ranges."""
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    avg_hsv = np.mean(hsv_roi.reshape(-1, 3), axis=0)
    h, s, v = avg_hsv
    
    # Define refined HSV ranges including Purple
    color_ranges = {
        "Red": [(0, 10, 100, 255, 100, 255), (160, 180, 100, 255, 100, 255)],
        "Orange": [(11, 25, 150, 255, 100, 255)],
        "Yellow": [(26, 35, 150, 255, 100, 255)],
        "Green": [(36, 85, 100, 255, 100, 255)],
        "Blue": [(86, 125, 100, 255, 100, 255)],
        "Indigo": [(126, 140, 100, 255, 100, 255)],
        "Violet": [(141, 160, 100, 255, 100, 255)],
        "Purple": [(125, 155, 100, 255, 50, 255)]  # Added purple
    }
    
    for color, ranges in color_ranges.items():
        for low_h, high_h, low_s, high_s, low_v, high_v in ranges:
            if low_h <= h <= high_h and low_s <= s <= high_s and low_v <= v <= high_v:
                return color
    return "Unknown"

def align_views(front_img, back_img):
    """Align front and back views using homography."""
    orb = cv2.ORB_create(nfeatures=2000)
    kp1, des1 = orb.detectAndCompute(front_img, None)
    kp2, des2 = orb.detectAndCompute(back_img, None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)[:50]
    
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    return H

def count_unique_fruits(front_path, back_path, model_path):
    """Count unique fruits across both views based on location and color."""
    model = YOLO(model_path)
    front_img = cv2.imread(front_path)
    back_img = cv2.imread(back_path)
    
    front_fruits = detect_fruits(front_path, model, "Front")
    back_fruits = detect_fruits(back_path, model, "Back")
    
    H = align_views(front_img, back_img)
    
    back_centers = np.float32([[f["bbox"][:2]] for f in back_fruits])
    transformed_centers = cv2.perspectiveTransform(back_centers, H).reshape(-1, 2) if len(back_centers) > 0 else []
    
    matched_back = set()
    unique_fruits = []
    
    for front in front_fruits:
        front_color = front["color"]
        matched = False
        
        for i, trans_center in enumerate(transformed_centers):
            if i not in matched_back and front_color == back_fruits[i]["color"]:
                matched_back.add(i)
                matched = True
                break
        
        if not matched:
            unique_fruits.append(front)
    
    for i in range(len(back_fruits)):
        if i not in matched_back:
            unique_fruits.append(back_fruits[i])
    
    print("\nUnique Fruits and Their Colors:")
    for fruit in unique_fruits:
        print(f"View: {fruit['view']}, Color: {fruit['color']}")
    
    return len(unique_fruits)

if __name__ == "__main__":
    front_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\front.jpg'
    back_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\back.jpg'
    model_path = r'C:\Users\saart\OneDrive\Desktop\best.pt'
    
    total_fruits = count_unique_fruits(front_path, back_path, model_path)
    print(f"\nTotal unique fruits detected: {total_fruits}")



image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\front.jpg: 480x640 5 2s, 3 3s, 205.5ms
Speed: 4.1ms preprocess, 205.5ms inference, 4.2ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\back.jpg: 480x640 8 2s, 3 3s, 263.7ms
Speed: 0.0ms preprocess, 263.7ms inference, 10.7ms postprocess per image at shape (1, 3, 480, 640)

Unique Fruits and Their Colors:
View: Front, Color: Blue
View: Front, Color: Blue
View: Front, Color: Blue
View: Back, Color: Unknown
View: Back, Color: Unknown
View: Back, Color: Unknown
View: Back, Color: Unknown
View: Back, Color: Unknown
View: Back, Color: Unknown

Total unique fruits detected: 9


In [8]:
from ultralytics import YOLO
import cv2
import numpy as np

def detect_fruits(img_path, model, view):
    """Detect fruits in an image, find their colors, and keep track of which view they belong to."""
    results = model.predict(img_path)[0]
    fruits = []
    img = cv2.imread(img_path)
    
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        center = ((x1 + x2) // 2, (y1 + y2) // 2)  # Restore center calculation
        fruit_roi = img[y1:y2, x1:x2]  # Extract fruit area
        color = detect_color(fruit_roi)  # Find the fruit color
        
        fruits.append({
            "view": view,  # Store if it's from the front or back image
            "bbox": (x1, y1, x2, y2),
            "center": center,  # Add center back
            "color": color
        })
    return fruits

def detect_color(roi):
    """Find the dominant color of a fruit using HSV color space."""
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    avg_hsv = np.mean(hsv_roi.reshape(-1, 3), axis=0)  # Calculate average HSV value
    h, s, v = avg_hsv
    
    # Define color ranges for detection
    color_ranges = {
        "Red": [(0, 10, 100, 255, 100, 255), (160, 180, 100, 255, 100, 255)],
        "Orange": [(11, 25, 150, 255, 100, 255)],
        "Yellow": [(26, 35, 150, 255, 100, 255)],
        "Green": [(36, 85, 100, 255, 100, 255)],
        "Blue": [(86, 125, 100, 255, 100, 255)],
        "Indigo": [(126, 140, 100, 255, 100, 255)],
        "Violet": [(141, 160, 100, 255, 100, 255)],
        "Purple": [(125, 155, 100, 255, 50, 255)]  # Added purple
    }
    
    for color, ranges in color_ranges.items():
        for low_h, high_h, low_s, high_s, low_v, high_v in ranges:
            if low_h <= h <= high_h and low_s <= s <= high_s and low_v <= v <= high_v:
                return color
    return "Unknown"

def align_views(front_img, back_img):
    """Aligns the front and back views using feature matching."""
    orb = cv2.ORB_create(nfeatures=2000)
    kp1, des1 = orb.detectAndCompute(front_img, None)
    kp2, des2 = orb.detectAndCompute(back_img, None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)[:50]  # Take best matches
    
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    return H

def count_unique_fruits(front_path, back_path, model_path):
    """Count the number of unique fruits seen from both front and back views."""
    model = YOLO(model_path)
    front_img = cv2.imread(front_path)
    back_img = cv2.imread(back_path)
    
    front_fruits = detect_fruits(front_path, model, "Front")
    back_fruits = detect_fruits(back_path, model, "Back")
    
    H = align_views(front_img, back_img)
    
    back_centers = np.float32([[f["center"]] for f in back_fruits])  # Use center instead of bbox
    transformed_centers = cv2.perspectiveTransform(back_centers, H).reshape(-1, 2) if len(back_centers) > 0 else []
    
    matched_back = set()
    unique_fruits = []
    
    for front in front_fruits:
        front_center = np.array(front["center"])
        front_color = front["color"]
        matched = False
        
        for i, trans_center in enumerate(transformed_centers):
            if i not in matched_back and front_color == back_fruits[i]["color"]:
                dist = np.linalg.norm(front_center - trans_center)
                if dist < 30:  # Keep reasonable distance threshold
                    matched_back.add(i)
                    matched = True
                    break
        
        if not matched:
            unique_fruits.append(front)
    
    for i in range(len(back_fruits)):
        if i not in matched_back:
            unique_fruits.append(back_fruits[i])
    
    print("\nUnique Fruits and Their Colors:")
    for fruit in unique_fruits:
        print(f"View: {fruit['view']}, Color: {fruit['color']}")
    
    return len(unique_fruits)

if __name__ == "__main__":
    front_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Front4.jpg'
    back_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Back4.jpg'
    model_path = r'C:\Users\saart\OneDrive\Desktop\best.pt'
    
    total_fruits = count_unique_fruits(front_path, back_path, model_path)
    print(f"\nTotal unique fruits detected: {total_fruits}")



image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Front4.jpg: 480x640 1 2, 1 3, 562.2ms
Speed: 15.9ms preprocess, 562.2ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Back4.jpg: 480x640 1 3, 359.2ms
Speed: 7.7ms preprocess, 359.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

Unique Fruits and Their Colors:
View: Front, Color: Unknown

Total unique fruits detected: 1


In [22]:
from ultralytics import YOLO
import cv2
import numpy as np

def detect_fruits(img_path, model, view):
    """Detect fruits in an image, find their centers, colors, and keep track of which view they belong to."""
    results = model.predict(img_path)[0]
    fruits = []
    img = cv2.imread(img_path)
    
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        center = ((x1 + x2) // 2, (y1 + y2) // 2)  # Calculate the center of the fruit
        fruit_roi = img[y1:y2, x1:x2]  # Extract fruit area
        color = detect_color(fruit_roi)  # Find the fruit color
        
        fruits.append({
            "view": view,  # Store if it's from the front or back image
            "center": center,  # Store the center of the fruit
            "color": color
        })
    return fruits

def detect_color(roi):
    """Find the dominant color of a fruit using HSV color space."""
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    avg_hsv = np.mean(hsv_roi.reshape(-1, 3), axis=0)  # Calculate average HSV value
    h, s, v = avg_hsv
    
    # Refined color ranges for detection
    color_ranges = {
        "Red": [(0, 10, 100, 255, 100, 255), (160, 180, 100, 255, 100, 255)],
        "Orange": [(11, 25, 150, 255, 100, 255)],
        "Yellow": [(26, 35, 150, 255, 100, 255)],
        "Green": [(36, 85, 100, 255, 100, 255)],
        "Blue": [(86, 125, 100, 255, 100, 255)],
        "Indigo": [(126, 140, 100, 255, 100, 255)],
        "Violet": [(141, 160, 100, 255, 100, 255)],
        "Purple": [(125, 155, 50, 255, 50, 255)]  # Adjusted range for better purple detection
    }
    
    for color, ranges in color_ranges.items():
        for low_h, high_h, low_s, high_s, low_v, high_v in ranges:
            if low_h <= h <= high_h and low_s <= s <= high_s and low_v <= v <= high_v:
                return color
    return "Unknown"

def align_views(front_img, back_img):
    """Aligns the front and back views using feature matching."""
    orb = cv2.ORB_create(nfeatures=2000)
    kp1, des1 = orb.detectAndCompute(front_img, None)
    kp2, des2 = orb.detectAndCompute(back_img, None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)[:50]  # Take best matches
    
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    return H

def count_unique_fruits(front_path, back_path, model_path):
    """Count the number of unique fruits seen from both front and back views."""
    model = YOLO(model_path)
    front_img = cv2.imread(front_path)
    back_img = cv2.imread(back_path)
    
    front_fruits = detect_fruits(front_path, model, "Front")
    back_fruits = detect_fruits(back_path, model, "Back")
    
    H = align_views(front_img, back_img)
    
    back_centers = np.float32([fruit["center"] for fruit in back_fruits]).reshape(-1, 1, 2)
    transformed_centers = cv2.perspectiveTransform(back_centers, H).reshape(-1, 2) if len(back_centers) > 0 else []
    
    matched_back = set()
    unique_fruits = []
    
    for front in front_fruits:
        front_color = front["color"]
        matched = False
        
        for i, trans_center in enumerate(transformed_centers):
            if i not in matched_back and front_color == back_fruits[i]["color"]:
                matched_back.add(i)
                matched = True
                break
        
        if not matched:
            unique_fruits.append(front)
    
    for i in range(len(back_fruits)):
        if i not in matched_back:
            unique_fruits.append(back_fruits[i])
    
    print("\nUnique Fruits and Their Colors:")
    for fruit in unique_fruits:
        print(f"View: {fruit['view']}, Color: {fruit['color']}")
    
    return len(unique_fruits)

if __name__ == "__main__":
    front_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\3\Front3.jpg'
    back_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\3\Back3.jpg'
    model_path = r'C:\Users\saart\OneDrive\Desktop\best.pt'
    
    total_fruits = count_unique_fruits(front_path, back_path, model_path)
    print(f"\nTotal unique fruits detected: {total_fruits}")



image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\3\Front3.jpg: 480x640 4 1s, 1 3, 377.6ms
Speed: 7.0ms preprocess, 377.6ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\3\Back3.jpg: 480x640 4 1s, 1 3, 242.8ms
Speed: 7.0ms preprocess, 242.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

Unique Fruits and Their Colors:
View: Front, Color: Unknown
View: Back, Color: Green

Total unique fruits detected: 2


In [10]:
from ultralytics import YOLO
import cv2
import numpy as np

def detect_fruits(img_path, model, view):
    """Detect fruits in an image, find their centers, colors, and keep track of which view they belong to."""
    results = model.predict(img_path)[0]
    fruits = []
    img = cv2.imread(img_path)
    
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        center = ((x1 + x2) // 2, (y1 + y2) // 2)
        fruit_roi = img[y1:y2, x1:x2]
        color = detect_color(fruit_roi)
        
        fruits.append({
            "view": view,
            "center": center,
            "color": color
        })
    return fruits

def detect_color(roi):
    """Find the dominant color using HSV masking and morphological operations."""
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    
    # Define color ranges with HSV values
    color_ranges = {
        "Red": [
            ([0, 100, 100], [10, 255, 255]),
            ([160, 100, 100], [180, 255, 255])
        ],
        "Orange": [
            ([11, 150, 100], [25, 255, 255])
        ],
        "Yellow": [
            ([26, 150, 100], [35, 255, 255])
        ],
        "Green": [
            ([36, 100, 100], [85, 255, 255])
        ],
        "Blue": [
            ([86, 100, 100], [124, 255, 255])
        ],
        "Purple": [
            ([125, 50, 50], [159, 255, 255])
        ]
    }

    max_pixels = 0
    dominant_color = "Unknown"
    
    for color, ranges in color_ranges.items():
        mask = np.zeros(hsv_roi.shape[:2], dtype=np.uint8)
        
        # Combine all ranges for the color
        for (lower, upper) in ranges:
            lower = np.array(lower, dtype=np.uint8)
            upper = np.array(upper, dtype=np.uint8)
            mask |= cv2.inRange(hsv_roi, lower, upper)
        
        # Clean up the mask
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
        
        # Calculate percentage of color pixels
        pixel_count = cv2.countNonZero(mask)
        if pixel_count > max_pixels and pixel_count > 0.1 * mask.size:
            max_pixels = pixel_count
            dominant_color = color
            
    return dominant_color

# Rest of the code remains unchanged
def align_views(front_img, back_img):
    """Aligns the front and back views using feature matching."""
    orb = cv2.ORB_create(nfeatures=2000)
    kp1, des1 = orb.detectAndCompute(front_img, None)
    kp2, des2 = orb.detectAndCompute(back_img, None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)[:50]
    
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    return H

def count_unique_fruits(front_path, back_path, model_path):
    """Count the number of unique fruits seen from both front and back views."""
    model = YOLO(model_path)
    front_img = cv2.imread(front_path)
    back_img = cv2.imread(back_path)
    
    front_fruits = detect_fruits(front_path, model, "Front")
    back_fruits = detect_fruits(back_path, model, "Back")
    
    H = align_views(front_img, back_img)
    
    back_centers = np.float32([fruit["center"] for fruit in back_fruits]).reshape(-1, 1, 2)
    transformed_centers = cv2.perspectiveTransform(back_centers, H).reshape(-1, 2) if len(back_centers) > 0 else []
    
    matched_back = set()
    unique_fruits = []
    
    for front in front_fruits:
        front_color = front["color"]
        matched = False
        
        for i, trans_center in enumerate(transformed_centers):
            if i not in matched_back and front_color == back_fruits[i]["color"]:
                matched_back.add(i)
                matched = True
                break
        
        if not matched:
            unique_fruits.append(front)
    
    for i in range(len(back_fruits)):
        if i not in matched_back:
            unique_fruits.append(back_fruits[i])
    
    print("\nUnique Fruits and Their Colors:")
    for fruit in unique_fruits:
        print(f"View: {fruit['view']}, Color: {fruit['color']}")
    
    return len(unique_fruits)

if __name__ == "__main__":
    front_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Front4.jpg'
    back_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Back4.jpg'
    model_path = r'C:\Users\saart\OneDrive\Desktop\best.pt'
    
    total_fruits = count_unique_fruits(front_path, back_path, model_path)
    print(f"\nTotal unique fruits detected: {total_fruits}")


image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Front4.jpg: 480x640 1 2, 1 3, 451.0ms
Speed: 7.6ms preprocess, 451.0ms inference, 7.1ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Back4.jpg: 480x640 1 3, 400.4ms
Speed: 8.1ms preprocess, 400.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

Unique Fruits and Their Colors:
View: Front, Color: Purple

Total unique fruits detected: 1


In [14]:
from ultralytics import YOLO
import cv2
import numpy as np

def detect_fruits(img_path, model, view):
    """Detect fruits in an image, find their centers, colors, and keep track of which view they belong to."""
    results = model.predict(img_path)[0]
    fruits = []
    img = cv2.imread(img_path)
    
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        center = ((x1 + x2) // 2, (y1 + y2) // 2)
        fruit_roi = img[y1:y2, x1:x2]
        color = detect_color(fruit_roi)
        
        fruits.append({
            "view": view,
            "center": center,
            "color": color
        })
    return fruits

def detect_color(roi):
    """Find the dominant color using HSV masking and morphological operations."""
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    
    # Define color ranges with HSV values
    color_ranges = {
        "Red": [
            ([0, 100, 100], [10, 255, 255]),
            ([160, 100, 100], [180, 255, 255])
        ],
        "Orange": [
            ([11, 150, 100], [25, 255, 255])
        ],
        "Yellow": [
            ([26, 150, 100], [35, 255, 255])
        ],
        "Green": [
            ([36, 100, 100], [85, 255, 255])
        ],
        "Blue": [
            ([86, 100, 100], [124, 255, 255])
        ],
        "Purple": [
            ([125, 50, 50], [159, 255, 255])
        ]
    }

    max_pixels = 0
    dominant_color = "Unknown"
    
    for color, ranges in color_ranges.items():
        mask = np.zeros(hsv_roi.shape[:2], dtype=np.uint8)
        
        # Combine all ranges for the color
        for (lower, upper) in ranges:
            lower = np.array(lower, dtype=np.uint8)
            upper = np.array(upper, dtype=np.uint8)
            mask |= cv2.inRange(hsv_roi, lower, upper)
        
        # Clean up the mask
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
        
        # Calculate percentage of color pixels
        pixel_count = cv2.countNonZero(mask)
        if pixel_count > max_pixels and pixel_count > 0.1 * mask.size:
            max_pixels = pixel_count
            dominant_color = color
            
    return dominant_color

# Rest of the code remains unchanged
def align_views(front_img, back_img):
    """Aligns the front and back views using feature matching."""
    orb = cv2.ORB_create(nfeatures=2000)
    kp1, des1 = orb.detectAndCompute(front_img, None)
    kp2, des2 = orb.detectAndCompute(back_img, None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)[:50]
    
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    return H

def count_unique_fruits(front_path, back_path, model_path):
    """Count the number of unique fruits seen from both front and back views."""
    model = YOLO(model_path)
    front_img = cv2.imread(front_path)
    back_img = cv2.imread(back_path)
    
    front_fruits = detect_fruits(front_path, model, "Front")
    back_fruits = detect_fruits(back_path, model, "Back")
    
    H = align_views(front_img, back_img)
    
    back_centers = np.float32([fruit["center"] for fruit in back_fruits]).reshape(-1, 1, 2)
    transformed_centers = cv2.perspectiveTransform(back_centers, H).reshape(-1, 2) if len(back_centers) > 0 else []
    
    matched_back = set()
    unique_fruits = []
    
    for front in front_fruits:
        front_color = front["color"]
        matched = False
        
        for i, trans_center in enumerate(transformed_centers):
            if i not in matched_back and front_color == back_fruits[i]["color"]:
                matched_back.add(i)
                matched = True
                break
        
        if not matched:
            unique_fruits.append(front)
    
    for i in range(len(back_fruits)):
        if i not in matched_back:
            unique_fruits.append(back_fruits[i])
    
    print("\nUnique Fruits and Their Colors:")
    for fruit in unique_fruits:
        print(f"View: {fruit['view']}, Color: {fruit['color']}")
    
    return len(unique_fruits)

if __name__ == "__main__":
    front_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\5\Front5.jpg'
    back_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\5\Back5.jpg'
    model_path = r'C:\Users\saart\OneDrive\Desktop\best.pt'
    
    total_fruits = count_unique_fruits(front_path, back_path, model_path)
    print(f"\nTotal unique fruits detected: {total_fruits}")


image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\5\Front5.jpg: 480x640 8 1s, 3 3s, 480.2ms
Speed: 6.5ms preprocess, 480.2ms inference, 8.3ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\5\Back5.jpg: 480x640 7 1s, 3 3s, 502.2ms
Speed: 6.5ms preprocess, 502.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

Unique Fruits and Their Colors:
View: Front, Color: Yellow

Total unique fruits detected: 1


In [20]:
from ultralytics import YOLO
import cv2
import numpy as np

def detect_fruits(img_path, model, view):
    """Detect fruits in an image, find their colors, and keep track of which view they belong to."""
    results = model.predict(img_path)[0]
    fruits = []
    img = cv2.imread(img_path)
    
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        center = ((x1 + x2) // 2, (y1 + y2) // 2)
        fruit_roi = img[y1:y2, x1:x2]
        color = detect_color(fruit_roi)  # Modified color detection
        
        fruits.append({
            "view": view,
            "bbox": (x1, y1, x2, y2),
            "center": center,
            "color": color
        })
    return fruits

def detect_color(roi):
    """Find dominant color using HSV masking."""
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    height, width = hsv.shape[:2]
    
    # Define color ranges (H_min, S_min, V_min), (H_max, S_max, V_max)
    color_masks = {
        "Red": [
            ((0, 100, 100), (10, 255, 255)),
            ((160, 100, 100), (180, 255, 255))
        ],
        "Orange": [((11, 150, 100), (25, 255, 255))],
        "Yellow": [((26, 150, 100), (35, 255, 255))],
        "Green": [((36, 100, 100), (85, 255, 255))],
        "Blue": [((86, 100, 100), (124, 255, 255))],
        "Purple": [((125, 50, 50), (159, 255, 255))]  # Wider purple range
    }

    dominant_color = "Unknown"
    max_pixels = 0
    
    for color_name, ranges in color_masks.items():
        combined_mask = np.zeros((height, width), dtype=np.uint8)
        
        # Combine all ranges for the color
        for (lower, upper) in ranges:
            lower = np.array(lower, dtype=np.uint8)
            upper = np.array(upper, dtype=np.uint8)
            mask = cv2.inRange(hsv, lower, upper)
            combined_mask = cv2.bitwise_or(combined_mask, mask)
        
        # Clean up the mask
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        cleaned_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel, iterations=1)
        cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_CLOSE, kernel, iterations=2)
        
        # Count valid pixels
        pixel_count = cv2.countNonZero(cleaned_mask)
        if pixel_count > max_pixels and pixel_count > 0.1 * (height * width):
            max_pixels = pixel_count
            dominant_color = color_name
            
    return dominant_color

def align_views(front_img, back_img):
    """Aligns the front and back views using feature matching."""
    orb = cv2.ORB_create(nfeatures=2000)
    kp1, des1 = orb.detectAndCompute(front_img, None)
    kp2, des2 = orb.detectAndCompute(back_img, None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)[:50]  # Take best matches
    
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    return H

def count_unique_fruits(front_path, back_path, model_path):
    """Count the number of unique fruits seen from both front and back views."""
    model = YOLO(model_path)
    front_img = cv2.imread(front_path)
    back_img = cv2.imread(back_path)
    
    front_fruits = detect_fruits(front_path, model, "Front")
    back_fruits = detect_fruits(back_path, model, "Back")
    
    H = align_views(front_img, back_img)
    
    back_centers = np.float32([[f["center"]] for f in back_fruits])  # Use center instead of bbox
    transformed_centers = cv2.perspectiveTransform(back_centers, H).reshape(-1, 2) if len(back_centers) > 0 else []
    
    matched_back = set()
    unique_fruits = []
    
    for front in front_fruits:
        front_center = np.array(front["center"])
        front_color = front["color"]
        matched = False
        
        for i, trans_center in enumerate(transformed_centers):
            if i not in matched_back and front_color == back_fruits[i]["color"]:
                dist = np.linalg.norm(front_center - trans_center)
                if dist < 30:  # Keep reasonable distance threshold
                    matched_back.add(i)
                    matched = True
                    break
        
        if not matched:
            unique_fruits.append(front)
    
    for i in range(len(back_fruits)):
        if i not in matched_back:
            unique_fruits.append(back_fruits[i])
    
    print("\nUnique Fruits and Their Colors:")
    for fruit in unique_fruits:
        print(f"View: {fruit['view']}, Color: {fruit['color']}")
    
    return len(unique_fruits)

if __name__ == "__main__":
    front_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\front.jpg'
    back_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\back.jpg'
    model_path = r'C:\Users\saart\OneDrive\Desktop\best.pt'
    
    total_fruits = count_unique_fruits(front_path, back_path, model_path)
    print(f"\nTotal unique fruits detected: {total_fruits}")



image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\front.jpg: 480x640 5 2s, 3 3s, 469.5ms
Speed: 6.5ms preprocess, 469.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\back.jpg: 480x640 8 2s, 3 3s, 423.4ms
Speed: 6.8ms preprocess, 423.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

Unique Fruits and Their Colors:
View: Front, Color: Unknown
View: Front, Color: Unknown
View: Back, Color: Unknown
View: Back, Color: Unknown
View: Back, Color: Unknown
View: Back, Color: Unknown
View: Back, Color: Unknown

Total unique fruits detected: 7


In [1]:
from ultralytics import YOLO
import cv2
import numpy as np

def detect_fruits(img_path, model, view):
    """Detect fruits in an image, find their colors, and keep track of which view they belong to."""
    results = model.predict(img_path)[0]
    fruits = []
    img = cv2.imread(img_path)
    
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        center = ((x1 + x2) // 2, (y1 + y2) // 2)
        fruit_roi = img[y1:y2, x1:x2]
        color = detect_color(fruit_roi)  # Modified color detection
        
        fruits.append({
            "view": view,
            "bbox": (x1, y1, x2, y2),
            "center": center,
            "color": color
        })
    return fruits

def detect_color(roi):
    """Find dominant color using HSV masking."""
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    height, width = hsv.shape[:2]
    
    # Define color ranges (H_min, S_min, V_min), (H_max, S_max, V_max)
    color_masks = {
        "Red": [
            ((0, 100, 100), (10, 255, 255)),
            ((160, 100, 100), (180, 255, 255))
        ],
        "Orange": [((11, 150, 100), (25, 255, 255))],
        "Yellow": [((26, 150, 100), (35, 255, 255))],
        "Green": [((36, 100, 100), (85, 255, 255))],
        "Blue": [((86, 100, 100), (124, 255, 255))],
        "Purple": [((125, 50, 50), (159, 255, 255))]  # Wider purple range
    }

    dominant_color = "Unknown"
    max_pixels = 0
    
    for color_name, ranges in color_masks.items():
        combined_mask = np.zeros((height, width), dtype=np.uint8)
        
        # Combine all ranges for the color
        for (lower, upper) in ranges:
            lower = np.array(lower, dtype=np.uint8)
            upper = np.array(upper, dtype=np.uint8)
            mask = cv2.inRange(hsv, lower, upper)
            combined_mask = cv2.bitwise_or(combined_mask, mask)
        
        # Clean up the mask
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        cleaned_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel, iterations=1)
        cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_CLOSE, kernel, iterations=2)
        
        # Count valid pixels
        pixel_count = cv2.countNonZero(cleaned_mask)
        if pixel_count > max_pixels and pixel_count > 0.1 * (height * width):
            max_pixels = pixel_count
            dominant_color = color_name
            
    return dominant_color

def align_views(front_img, back_img):
    """Aligns the front and back views using feature matching."""
    orb = cv2.ORB_create(nfeatures=2000)
    kp1, des1 = orb.detectAndCompute(front_img, None)
    kp2, des2 = orb.detectAndCompute(back_img, None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)[:50]  # Take best matches
    
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    return H

def count_unique_fruits(front_path, back_path, model_path):
    """Count the number of unique fruits seen from both front and back views."""
    model = YOLO(model_path)
    front_img = cv2.imread(front_path)
    back_img = cv2.imread(back_path)
    
    front_fruits = detect_fruits(front_path, model, "Front")
    back_fruits = detect_fruits(back_path, model, "Back")
    
    H = align_views(front_img, back_img)
    
    back_centers = np.float32([[f["center"]] for f in back_fruits])  # Use center instead of bbox
    transformed_centers = cv2.perspectiveTransform(back_centers, H).reshape(-1, 2) if len(back_centers) > 0 else []
    
    matched_back = set()
    unique_fruits = []
    
    for front in front_fruits:
        front_center = np.array(front["center"])
        front_color = front["color"]
        matched = False
        
        for i, trans_center in enumerate(transformed_centers):
            if i not in matched_back and front_color == back_fruits[i]["color"]:
                dist = np.linalg.norm(front_center - trans_center)
                if dist < 30:  # Keep reasonable distance threshold
                    matched_back.add(i)
                    matched = True
                    unique_fruits.append({"view": "Both", "color": front_color})
                    break
        
        if not matched:
            unique_fruits.append(front)
    
    for i in range(len(back_fruits)):
        if i not in matched_back:
            unique_fruits.append(back_fruits[i])
    
    print("\nUnique Fruits and Their Colors:")
    for fruit in unique_fruits:
        print(f"View: {fruit['view']}, Color: {fruit['color']}")
    
    return len(unique_fruits)

if __name__ == "__main__":
    front_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\5\Front5.jpg'
    back_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\5\Back5.jpg'
    model_path = r'C:\Users\saart\OneDrive\Desktop\best.pt'
    
    total_fruits = count_unique_fruits(front_path, back_path, model_path)
    print(f"\nTotal unique fruits detected: {total_fruits}")



image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\5\Front5.jpg: 480x640 8 1s, 3 3s, 466.2ms
Speed: 7.0ms preprocess, 466.2ms inference, 4.1ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\5\Back5.jpg: 480x640 7 1s, 3 3s, 414.3ms
Speed: 3.5ms preprocess, 414.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

Unique Fruits and Their Colors:
View: Both, Color: Green
View: Both, Color: Green
View: Both, Color: Green
View: Both, Color: Yellow
View: Both, Color: Yellow
View: Both, Color: Yellow
View: Both, Color: Yellow
View: Front, Color: Yellow
View: Front, Color: Yellow
View: Both, Color: Yellow
View: Front, Color: Yellow
View: Back, Color: Yellow
View: Back, Color: Yellow

Total unique fruits detected: 13
